In [ ]:
!pip install tqdm

In [77]:
import pandas as pd 
import numpy as np
from scipy import stats
from scipy.stats import ranksums
from tqdm import tqdm
np.random.seed(12345678)
filename = 'AMI_GSE66360_series_matrix.csv'


100%|██████████| 1000000/1000000 [00:00<00:00, 3096311.64it/s]


In [66]:


labels = pd.read_csv(filename,skiprows=58,nrows=1).to_numpy()
data = pd.read_csv(filename,skiprows=60) 
print(data.shape)
print(labels)
data.head()
#a.1 - 54674 genes are profiled
#a.2 - 99 samples total
#a.3 - 2 classes, Healthy - Myocradial



(54675, 100)
[['Class' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'M' 'M' 'M'
  'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
  'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
  'H' 'H' 'H' 'H' 'H' 'H' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
  'M' 'M' 'M' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'M' 'M'
  'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M']]


'Class'

In [6]:
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))
#a.3 - 50 H , 49 - M

{'Class': 1, 'H': 50, 'M': 49}

In [7]:
data.dropna().shape
#a.4 - 54628

(54628, 100)

In [9]:
#a.5
data_np = data.dropna().to_numpy()
#print(data_np.shape)
results = pd.read_csv(filename,skiprows=58,nrows=1).to_numpy()
rand_genes_idx = np.random.randint(low=0,high=data_np.shape[0]-1, size=20)
#print(rand_idx)

B.1 - 2450

In [10]:
x = ((99+1)/2 * 49)
print(x)

2450.0


B.2 - maximal value is the sum of 51 to 99 where the gene represent H/M solely, meaning: 3675

In [40]:
sum = 0
for i in range(51,100):
    sum += i
print(sum)

3675


B.3 - under the null model, we equal probability for every configuration, and 49 samples that contain M, we need to choose 49 from 99 samples, meaning $P[RS(g))=c] = \dfrac{1}{{99 \choose 49}}$

B.4

$P[RS(g))=c-1] = \dfrac{1}{{99 \choose 49}}$ it has the same probability as B.3 since having the rank lowered by 1 meaning we swapped 2 ranks, the 51th and the 50th, so there is only one option, and we have the same as B.3
 
 $P[RS(g))=c-2] = \dfrac{1}{{99 \choose 49}} + \dfrac{1}{{99 \choose 49}}$ we have two permutations that can yield a c-2 result, is by switching 51th rank with 49th or switchin 50th rank with 52th rank.
 

In [117]:
data.head()

,ID_REF,GSM1620819,GSM1620820,GSM1620821,GSM1620822,GSM1620823,GSM1620824,GSM1620825,GSM1620826,GSM1620827,...,GSM1620908,GSM1620909,GSM1620910,GSM1620911,GSM1620912,GSM1620913,GSM1620914,GSM1620915,GSM1620916,GSM1620917
0,1007_s_at,5.866378,4.948736,5.148384,5.064419,5.017833,5.116154,5.431328,5.235271,5.773528,...,5.419482,5.057716,5.996493,5.343133,5.558892,6.472517,5.678816,5.653286,6.013841,5.465334
1,1053_at,8.225790,7.525865,7.764637,7.198461,7.831450,7.203592,7.694551,7.760259,8.279814,...,7.226348,7.105538,5.354105,8.271500,5.963346,8.261422,6.925753,7.918424,7.442701,9.225221
2,117_at,6.179732,6.628137,5.859151,5.974135,6.793079,6.435229,6.320546,6.483590,6.387779,...,6.789501,7.441940,7.026929,6.003973,7.474326,6.367759,6.835756,7.577035,7.147417,6.481128
3,121_at,6.179478,6.582889,6.602135,6.545905,5.911542,6.285420,6.562316,6.345888,6.272806,...,6.513437,6.317523,7.413453,6.809103,5.949091,6.440978,6.955883,6.499215,6.587065,6.897591
4,1255_g_at,2.792048,2.695714,2.580181,2.712124,2.723687,2.706684,2.674747,2.635150,2.696649,...,2.924658,2.714103,3.635945,2.710944,2.979381,3.730139,2.914868,3.013901,2.729320,3.151449


### Question C

we'll find all the genes with one-sided wrs and ttest below 0.07
For these genes we'll decide whether they are over or under expressed in M label samples according to the sum of their rank. Low rank == underexpresed, high rank == overexpressed
Their number will just be lower than , then simple using ttest and wrs directly should be ok because they will be ranked up at the list when we sort it in an asc order. 


In [131]:

labels_boolean = []
# Skipping the first item which is just class keyword
for l in labels[0][1:]:
    if l =='H':
        labels_boolean.append(False)
    else:
        labels_boolean.append(True)


# print(labels_boolean)
# print(np.bitwise_not(labels_boolean))

ttest_underexpressed = []
wrs_underexpressed = [] 

for gene in tqdm(range(data.shape[0])):
#   Testing underexpressed
    M_labels = data.iloc[gene,1:][labels_boolean]
    H_labels = data.iloc[gene,1:][np.bitwise_not(labels_boolean)]
    one_sided_ttest = stats.ttest_ind(M_labels,H_labels)[1] /2 
    one_sided_wrs = ranksums(M_labels,H_labels)[1] /2 

    over_under = list(zip(data.iloc[gene,1:], labels_boolean))
    over_under.sort(key = lambda x: x[0])
    expression_level = 0
    for i in range(len(over_under)):
        if over_under[i][1]:
            expression_level += i
            
    if one_sided_ttest < 0.07:
        ttest_underexpressed.append({'gene_index': gene,'gene_name':  data.iloc[gene,0], 'ttest_value': one_sided_ttest, 'expression_level': expression_level})
    if one_sided_wrs < 0.07:
        wrs_underexpressed.append({'gene_index': gene,'gene_name':  data.iloc[gene,0], 'wrs_value': one_sided_wrs, 'expression_level': expression_level})
        


100%|██████████| 54675/54675 [01:34<00:00, 579.07it/s]


In [138]:
over_expressed_genes_ttest = 0
for res in ttest_underexpressed:
    if res['expression_level'] > 2525:
        over_expressed_genes_ttest += 1

print('number of over expressed genes in ttest: ', over_expressed_genes_ttest)
print('number of under expressed genes in ttest: ', len(ttest_underexpressed) - over_expressed_genes_ttest)
print('================')        
over_expressed_genes_wrs = 0
for res in wrs_underexpressed:
    if res['expression_level'] > 2525:
        over_expressed_genes_wrs += 1

print('number of over expressed genes in wrs: ',over_expressed_genes_wrs)
print('number of under expressed genes in wrs: ',len(wrs_underexpressed) - over_expressed_genes_wrs)

number of over expressed genes in ttest:  8430
number of under expressed genes in ttest:  8726
number of over expressed genes in wrs:  10800
number of under expressed genes in wrs:  10675


### Question D

